Machine Learning Assignment 2
Active Learning

Group Number : 29

Group Members:
Aquib Junaid Razack (2017A2PS0704P)
Shyam Sundar P I (2016A4PS0378P)
Praveen Ravirathinam (2017A7PS1174P)

Wine Classification
Question 3

# Setting up the environment

In [1]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_wine
from sklearn.preprocessing import MinMaxScaler
import warnings
warnings.filterwarnings('ignore')

In [2]:
from sklearn.model_selection import train_test_split

In [3]:
from sklearn.cluster import KMeans

In [4]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Preprocessing the data

In [5]:
wine = load_wine()
train = pd.DataFrame(wine.data, columns=wine.feature_names)
target = pd.DataFrame(wine.target, columns = ['Class'])
scal = MinMaxScaler()
train = scal.fit_transform(train)
train = pd.DataFrame(train, columns = wine.feature_names)
#Splitting into train and test
X_train, X_test, y_train, y_test = train_test_split(train, target, test_size = 0.20, random_state = 69)

# Clustering using K-means

In [6]:
kmeans = KMeans(n_clusters = 3, random_state = 69)
kmeans.fit(X_train)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=3, n_init=10, n_jobs=None, precompute_distances='auto',
    random_state=69, tol=0.0001, verbose=0)

In [7]:
X_train.shape

(142, 13)

# Dividing the dataset based on clusters

In [8]:
X_train['Cluster number'] = pd.DataFrame(kmeans.labels_).values

In [9]:
c_zero = X_train[X_train['Cluster number'] == 0]
c_one = X_train[X_train['Cluster number'] == 1]
c_two = X_train[X_train['Cluster number'] == 2]

# Assigning labels based on predictions of random points in clusters

In [10]:
oracle_c_zero = y_train.loc[c_zero.index,:]
rand = np.random.randint(0,c_zero.shape[0],15)
c_zero_rand = c_zero.iloc[rand,:]
label = oracle_c_zero.groupby('Class')['Class'].count().argmax()
c_zero['Class'] = np.full((c_zero.shape[0],1),label)

In [11]:
oracle_c_one = y_train.loc[c_one.index,:]
rand = np.random.randint(0,c_one.shape[0],15)
c_one_rand = c_one.iloc[rand,:]
label = oracle_c_one.groupby('Class')['Class'].count().argmax()
c_one['Class'] = np.full((c_one.shape[0],1),label)

In [12]:
oracle_c_two = y_train.loc[c_two.index,:]
rand = np.random.randint(0,c_two.shape[0],15)
c_two_rand = c_two.iloc[rand,:]
label = oracle_c_two.groupby('Class')['Class'].count().argmax()
c_two['Class'] = np.full((c_two.shape[0],1),label)

In [13]:
new_X_train = c_zero.append(c_one).append(c_two)

In [14]:
new_y_train = pd.DataFrame(new_X_train.loc[:,'Class'])

In [15]:
new_X_train = new_X_train.drop(columns = ['Class','Cluster number'])

# Training with the labelled points

In [16]:
rand = np.random.randint(0,new_X_train.shape[0],new_X_train.shape[0])

In [17]:
new_X_train = new_X_train.iloc[rand,:]
new_y_train = new_y_train.iloc[rand,:]

In [18]:
logit = LogisticRegression()
logit.fit(new_X_train,new_y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [19]:
pred = logit.predict(X_test)
accuracy_score(pred, y_test)

0.9166666666666666